In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 03: Agent-as-Tool Collaboration Pattern

This notebook demonstrates the **Agent-as-Tool** collaboration paradigm, where one agent uses another specialized agent as a tool. The financial assistant from Lecture 01 is enhanced with a `stock_report` tool that internally delegates to the Lecture 02 stock analysis agent.

## How Agent-as-Tool Works

This extends the ReAct pattern from Lecture 01 by adding multiple tool options. The agent now chooses between different tools based on the type of information needed:

```mermaid
graph TD
    Start([👤 User Question 👤]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| ToolChoice{Which<br/>Tool?}
    
    ToolChoice -->|General Info| WebSearch[🔧 Web Search&nbsp;<br/>Tool]
    ToolChoice -->|Stock Analysis| StockReport[🔧 Stock Report Tool&nbsp;<br/>Agent-as-Tool]
    
    WebSearch --> Results[📊 Receive<br/>Tool Results]
    StockReport -->|Delegates to<br/>Stock Analyzer Agent| StockResults[📊 Receive<br/>StockReport Object]
    StockResults --> Results
    
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer&nbsp;<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style ToolChoice fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style WebSearch fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockReport fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style StockResults fill:#ffebee,stroke:#b71c1c,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Differences from Lecture 01:**
- **Multiple Tools**: Agent now has 2 tools to choose from (web_search and stock_report)
- **Tool Selection**: Additional decision node for choosing the appropriate tool
- **Agent-as-Tool**: The stock_report tool is actually another agent (from Lecture 02) that runs independently
- **Structured Output**: Stock analysis returns typed `StockReport` objects instead of raw text

**Key Benefits:**
- **Modularity**: Specialized agents can be reused as tools by other agents
- **Separation of Concerns**: Each agent focuses on its specific domain
- **Type Safety**: Agents return structured Pydantic models ensuring data consistency
- **Reusability**: The stock analysis agent can be used standalone or as a tool


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import Tool
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging

# Initialize logging
setup_logging()

In [3]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="tool-delegation-lecture03",
    start_message="🚀 Lecture 03 - Tool Delegation Notebook Started",
)

In [4]:
# Define the StockReport model inline for self-contained presentation
from typing import List, Optional
from pydantic import BaseModel, Field
from IPython.display import Markdown, display


class StockReport(BaseModel):
    """Comprehensive stock analysis report."""

    symbol: str = Field(description="Stock symbol (e.g., AAPL, TSLA)")
    company_name: str = Field(description="Full company name")
    current_price: Optional[float] = Field(
        description="Current stock price", default=None
    )
    price_change: Optional[float] = Field(
        description="Price change from previous close", default=None
    )
    price_change_percent: Optional[float] = Field(
        description="Price change percentage", default=None
    )

    # Analysis sections
    executive_summary: str = Field(description="Brief executive summary of the stock")
    recent_news: List[str] = Field(
        description="Key recent news items affecting the stock", default_factory=list
    )
    financial_highlights: List[str] = Field(
        description="Key financial metrics and highlights", default_factory=list
    )
    market_sentiment: str = Field(description="Overall market sentiment analysis")
    risk_factors: List[str] = Field(
        description="Key risk factors to consider", default_factory=list
    )
    recommendation: str = Field(description="Investment recommendation with reasoning")

    # Metadata
    analysis_date: str = Field(description="Date when analysis was performed")
    data_sources: List[str] = Field(
        description="Sources used for the analysis", default_factory=list
    )

In [5]:
# Define helper functions and the analyze_stock agent inline
import logging
from datetime import datetime

logger = logging.getLogger(__name__)


def get_current_date() -> str:
    """Get current date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")


def validate_stock_symbol(symbol: str) -> str:
    """Validate and normalize stock symbol."""
    if not symbol:
        raise ValueError("Stock symbol cannot be empty")

    # Convert to uppercase and remove whitespace
    normalized = symbol.strip().upper()

    # Basic validation - should be 1-5 characters, alphanumeric
    if not normalized.isalnum() or len(normalized) > 5:
        raise ValueError(f"Invalid stock symbol format: {symbol}")

    return normalized


# Create the stock analysis agent (this is what was in lecture02)
stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockReport,
    system_prompt=f""""
You are an expert financial analyst specializing in stock analysis and market research.

IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date

Your task is to create comprehensive stock reports by gathering and analyzing current information about publicly traded companies.

CAPABILITIES:
- You have access to a web search tool that can find real-time information about stocks, companies, and market conditions
- You can search for financial news, earnings reports, analyst opinions, and market data
- You should gather information from multiple reliable sources

ANALYSIS FRAMEWORK:
When analyzing a stock, you should:

1. **Company Overview**: Get basic company information, business model, and recent developments
2. **Financial Performance**: Look for recent earnings, revenue trends, and key financial metrics
3. **Market Position**: Research competitive landscape and market share
4. **Recent News**: Find recent news that could impact the stock price
5. **Analyst Opinions**: Look for professional analyst ratings and price targets
6. **Risk Assessment**: Identify potential risks and challenges
7. **Market Sentiment**: Gauge overall market sentiment toward the stock

SEARCH STRATEGY:
- Use multiple targeted searches to gather comprehensive information
- Search for recent news (last 3-6 months) for current relevance
- Look for both positive and negative information to provide balanced analysis
- Verify information from multiple sources when possible
- Focus on reputable financial news sources and official company communications

OUTPUT REQUIREMENTS:
- Provide a structured analysis following the StockReport format
- Include specific data points when available (prices, percentages, dates)
- Cite your sources and indicate when information is current
- Be objective and balanced in your analysis
- Clearly distinguish between facts and opinions/projections
- If certain information is not available, acknowledge this rather than speculating

Remember: Your analysis should be thorough, factual, and useful for investment decision-making.
    """.strip(),
)


async def analyze_stock(ctx: RunContext, symbol: str) -> StockReport:
    """
    Analyze a stock and generate a comprehensive report.

    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')

    Returns:
        StockReport: Comprehensive analysis report
    """
    # Validate and normalize the stock symbol
    normalized_symbol = validate_stock_symbol(symbol)

    logger.info(f"Starting stock analysis for: {normalized_symbol}")

    # Create the analysis prompt
    prompt = f"""
Please analyze the stock {normalized_symbol} and provide a comprehensive report.

The analysis date should be: {get_current_date()}
    """.strip()

    try:
        # Run the agent analysis
        result = await stock_analysis_agent.run(prompt)

        logger.info(f"Stock analysis completed for {normalized_symbol}")
        logger.debug(f"Analysis result: {result.output}")

        return result.output

    except Exception as e:
        logger.error(f"Error analyzing stock {normalized_symbol}: {str(e)}")
        raise

In [ ]:
# Create a tool that delegates to the lecture02 stock analysis agent
stock_report_tool = Tool(
    analyze_stock,
    description="Get a detailed stock analysis report for a stock symbol (e.g., AAPL, TSLA).",
)

# Create enhanced financial assistant with both web_search and stock_report tools
enhanced_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, stock_report_tool],  # Now has 2 tools
    system_prompt=f"""
You are a knowledgeable financial assistant.

IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date

When answering questions:
1. Use the web_search tool for general financial information
2. Use the stock_report tool when users ask for detailed analysis of specific stocks
3. Combine information from both sources with your financial knowledge
4. Provide clear, actionable advice
5. Include appropriate disclaimers for investment advice

Keep your responses informative but concise.
""",
)

In [7]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** What is the current price of Bitcoin?

---

As of January 5, 2026, the current price of Bitcoin (BTC) is approximately \$92,655.58 USD. Please note that cryptocurrency prices can be volatile and may change rapidly. Always verify prices on your preferred exchange before making any trades or investments.

In [8]:
# Ask for detailed stock analysis (uses stock_report tool)
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

2026-01-05 20:08:30 - __main__ - INFO - Starting stock analysis for: AAPL
2026-01-05 20:08:41 - __main__ - INFO - Stock analysis completed for AAPL


**Question:** Can you give me a detailed analysis of Apple stock (AAPL)?

---

Here’s a detailed analysis of Apple, Inc. (AAPL) as of January 5, 2026:

Stock Overview
- Current Price: \$271.01 (down -0.31% today)
- Market Cap: ~\$4.03 trillion
- P/E Ratio: ~38 (premium versus tech peers)
- Dividend Yield: 0.52%

Business & Financial Highlights (Q4 FY2025)
- Revenue: \$102.5B (+8% YoY; record quarter)
- EPS: \$1.85; Net Income: \$27.47B (+8% YoY)
- iPhone revenue: \$49.02B (+6% YoY)
- Services revenue: \$28.75B (all-time high)
- Strong cash reserves (\$3.70/share)

Key Trends & Catalysts
- iPhone 17 sales hit record highs, supporting growth.
- Services (App Store, iCloud, Apple Care, etc.) is growing rapidly.
- Anticipation for 'Apple Intelligence' AI platform; some delays for core AI upgrades (e.g., Siri).
- Strong demand in China has boosted holiday forecasts and global share.

Recent News
- Regulators (EU, Japan, Brazil) are increasing scrutiny on App Store policies; Apple is opening iOS to third-party app stores in several regions.
- AI leadership changes and delays to Apple Intelligence product suite.

Market Sentiment & Analyst Views
- Sentiment: Moderately bullish, with a consensus 'Moderate Buy' rating.
- Average Analyst Target: ~\$299.50 (≈10% upside), with bullish targets near \$315.
- Apple remains a top pick for AI innovation and long-term sector leadership, but headline risk from regulation is rising.

Risks
- Significant regulatory and legal exposure (App Store economics, antitrust).
- Delays and competitive risk in AI innovation versus Microsoft and Google.
- Valuation is high, so shares may be sensitive to sector pullbacks or multiple compression.
- Hardware margin pressure from tariffs and supply chain issues.

Recommendation (as of 2026-01-05)
Apple remains a core long-term holding for most portfolios, given its profitability, innovation pipeline, and industry dominance. Consider adding or holding AAPL if you have a multi-year horizon, but monitor for regulatory news and possible short-term volatility. If you are risk-averse, consider exposure size due to Apple's high valuation and potential headline shocks.

Disclaimer: This analysis does not constitute personalized investment advice. Always consult with a financial advisor before making investment decisions. 

Would you like trend charts, valuation comparisons, or insights on portfolio allocation for Apple?

In [9]:
# Try another stock analysis
question = "Find 3 trending stocks and provide an investor report for them"
result = await enhanced_financial_agent.run(question)
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
display(Markdown(result.output.replace("$", "\\$")))

2026-01-05 20:08:51 - __main__ - INFO - Starting stock analysis for: NVDA
2026-01-05 20:08:51 - __main__ - INFO - Starting stock analysis for: IREN
2026-01-05 20:08:51 - __main__ - INFO - Starting stock analysis for: NEM
2026-01-05 20:09:04 - __main__ - INFO - Stock analysis completed for IREN
2026-01-05 20:09:06 - __main__ - INFO - Stock analysis completed for NVDA
2026-01-05 20:09:06 - __main__ - INFO - Stock analysis completed for NEM


**Question:** Find 3 trending stocks and provide an investor report for them

---

Here are 3 trending stocks in January 2026, along with concise investor reports:

---

### 1. NVIDIA Corporation (NVDA)
- **Sector:** Semiconductors / AI Hardware
- **Price:** \$190.53 (+1.26% today)
- **Summary:** NVIDIA leads the global GPU and AI hardware market, reporting record growth with FY2025 revenue at \$130.5B (+114% YoY). Key deals and partnerships (Groq, AWS, TSMC) reinforce its AI leadership. Order backlog exceeds \$500B for 2026, and sentiment is strongly bullish; average 12-month price target is \$250–\$260.
- **Highlights:**
  - EPS Q3 2025: \$1.24 (beat); next quarter est. \$1.45
  - Market cap: ~\$4.6T
  - Gross margin: >70%
- **Risks:** High valuation (P/E ~46x), AI investment cycle risk, regulatory uncertainty, insider selling, competitive pressure.
- **Action:** **Buy** for long-term growth and AI exposure. Be mindful of short-term volatility and valuation risks.

---

### 2. Iris Energy Limited (IREN)
- **Sector:** Bitcoin Mining / AI Data Centers
- **Price:** \$45.00 (+5.4% today)
- **Summary:** IREN is a fast-growing Bitcoin miner and AI data center provider. FY25 saw triple-digit growth; their entry into AI cloud services is boosting outlook. Recent deals (Microsoft, major Q1 contracts) and \$3.4B projected AI cloud revenue for 2026 position it as a unique dual-sector growth play.
- **Highlights:**
  - Revenue Q4 FY25: +164.9% YoY; net income +400%
  - Profit margin: ~76%; ROE: ~26%
  - Aggressive capex phase; market cap: \$14B
- **Risks:** Bitcoin price volatility, aggressive AI projections, heavy capex, regulatory risk.
- **Action:** **Buy** for growth-focused investors seeking exposure to both Bitcoin and AI sector growth. Accept sector volatility and execution risk.

---

### 3. Newmont Corporation (NEM)
- **Sector:** Gold Mining
- **Price:** \$101.22 (+1.37% today)
- **Summary:** Newmont, the world’s largest gold miner, is surging on record gold prices (\$4,500/oz) and post-acquisition cost synergies. Q3 FCF was \$1.6B; 2025 revenue was \$21.5B and is forecasted to exceed \$27B in 2026. Consensus is bullish, though shares may be “priced for perfection” after a 110% rally in 2025.
- **Highlights:**
  - Q3 2025 EPS: \$1.71 (beat)
  - Buyback: \$3B program, strong balance sheet
  - Positive cash flow, expanded assets (Newcrest deal)
- **Risks:** Gold price sensitivity, potential for correction, integration risk, sector regulatory/environmental risk.
- **Action:** **Accumulate** (take partial profits if overweight) for gold/inflation hedge; **Buy** for long-term commodity/currency hedge.

---

**Disclaimer:** These reports are for informational purposes only and do not constitute individualized financial advice. Always conduct further research and consider your risk tolerance before investing.